In [1]:
import torch
import torch.nn as nn
from merlin import QuantumLayer, ComputationSpace, LexGrouping
from merlin.builder import CircuitBuilder

In [ ]:
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import perceval as pcvl
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from merlin.algorithms.kernels import FeatureMap, FidelityKernel, KernelCircuitBuilder

torch.manual_seed(0)
np.random.seed(0)

In [ ]:
import pandas as pd
from sklearn.preprocessing import  MinMaxScaler

import pandas as pd



def load_data(file_path: str) -> pd.DataFrame:
    print(f"Loading data from {file_path}")
    return pd.read_csv(file_path)


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:

    X = df.iloc[:, 1:-1].values
    y = df.iloc[:, -1].values
    print('X = ', X)

    sc_X = MinMaxScaler()
    X_train = sc_X.fit_transform(X)
    print("X train = ", X_train)
            
    print("Preprocessing complete.")

    return X_train,y

def save_cleaned_data(df: pd.DataFrame, output_path: str) -> None:
    print(f"\nSaving cleaned data to {output_path}")
    df= pd.DataFrame(df, columns=['income','credit_utilization','payment_history','num_open_accounts','debt_to_income','loan_amount'])
    
    df.to_csv(output_path, index=False)
    print("Done.")


input_path = "data/credit_train.csv"
output_path = "data/credit_train_cleaned.csv"



df = load_data(input_path)
# inspect_data(df)
df_cleaned,y_train = preprocess_data(df)
# inspect_data(df_cleaned)
save_cleaned_data(df_cleaned, output_path)


test_input_path = "data/credit_test.csv"
test_output_path = "data/credit_test_cleaned.csv"

df_test = load_data(test_input_path)
# inspect_data(df)
df_test_cleaned,y_test = preprocess_data(df_test)
# inspect_data(df_cleaned)
save_cleaned_data(df_test_cleaned, test_output_path)

# ============================================================
# 1. Tensor conversions & DataLoader
# ============================================================
def convert_dataset_to_tensor(x_train, x_test, y_train, y_test):
    x_train = torch.tensor(x_train, dtype=torch.float32)
    x_test = torch.tensor(x_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)
    return x_train, x_test, y_train, y_test


def convert_tensor_to_loader(x_train, y_train, batch_size=6):
    train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True
    )
    return train_loader



x_train_np=df_cleaned
y_train_np=y_train

x_test_np=df_test_cleaned
y_test_np=y_test
# Number of classes inferred from training labels
num_classes = int(len(np.unique(y_train_np)))

# Convert to tensors
x_train, x_test, y_train, y_test = convert_dataset_to_tensor(
    x_train_np, x_test_np, y_train_np, y_test_np
)


# Build DataLoader
#train_loader = convert_tensor_to_loader(x_train, y_train, batch_size=6)


In [ ]:
print(len(x_test_np))

In [ ]:
kernel = FidelityKernel.simple(
    input_size=6,
    n_modes=6,
    shots=0,  # exact probabilities
    no_bunching=False,
    dtype=torch.float32,
    device=torch.device("cpu"),
)

K_train = kernel(x_train)
K_test = kernel(x_test, x_train)

print("Train Gram shape:", K_train.shape)
print("Test Gram shape:", K_test.shape)

In [ ]:
svc = SVC(kernel="precomputed")
svc.fit(K_train.detach().numpy(), y_train)
train_accuracy = svc.score(K_train.detach().numpy(), y_train)
print(f"SVM train accuracy (precomputed kernel): {train_accuracy:.3f}")
test_accuracy = svc.score(K_test.detach().numpy(), y_test)
print(f"SVM accuracy (precomputed kernel): {test_accuracy:.3f}")

In [ ]:
#AUC
#F1
#Recall